# Conexiones e importaciones

In [82]:
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
load_dotenv() #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)

In [83]:
#Spotify

mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

In [84]:
#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

# Spotify

In [63]:
generos = ["flamenco", "latin", "jazz", "rock"]
año = 2010
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
datos_artistas = {} 
artistas_procesados = set()

print("Buscando canciones del año", año)

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        



        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2010
Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 0 canciones
- latin: 1618 canciones
- jazz: 3249 canciones
- rock: 4257 canciones
Total de canciones encontradas: 9124


In [64]:
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
todas_las_canciones_df

,nombre,artista,album,genero,año
0,The Christmas Gift,Morat,The Christmas Gift,latin,2010
1,El Demonio De La Tinta,Kapo,El Demonio De La Tinta,latin,2010
2,Ain't Stoppin Me - The Official Rampage Jackso...,Kapo,Ain't Stoppin Me - The Official Rampage Jackso...,latin,2010
3,Ain't Stoppin Me- The Official Rampage Jackson...,Kapo,Ain't Stoppin Me - The Official Rampage Jackso...,latin,2010
4,Ain't Stoppin Me - Instrumental,Kapo,Ain't Stoppin Me - The Official Rampage Jackso...,latin,2010
...,...,...,...,...,...
9119,Married In Vegas,Maná,The Chase,rock,2010
9120,Renegade,Maná,The Chase,rock,2010
9121,The Chase,Maná,The Chase,rock,2010
9122,Better Cause of You,Maná,The Chase,rock,2010


In [65]:
todas_las_canciones_df.to_csv("canciones_2010.csv", index=False)

# Last FM

In [61]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [66]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [ ]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            tags = [tag['name'] for tag in artista_info.get('tags', {}).get('tag', [])]
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = todas_las_canciones_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)

    # 6. Unir los datos de Last.fm al DataFrame principal de canciones
    df_final = pd.merge(
        todas_las_canciones_df,
        datos_lastfm_df, # ¡Usamos el DF recién creado!
        on='artista',
        how='left'
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 83

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [79]:
datos_lastfm_df

,artista,tags_lastfm,bio_resumen,listeners,consulta_exitosa
0,Morat,"colombian, latin pop, folk pop, latino, pop",Morat is a Colombian band formed in the countr...,291934,True
1,Kapo,"acoustic, Colombia",The band Kapo was formed in 2006 by Joe Muller...,127043,True
2,Beéle,"Colombia, latin, pop, hip hop, male vocalists","Brandon De Jesús López Orozco, mejor conocido ...",188316,True
3,Shakira,"pop, latin, female vocalists, spanish, rock",Shakira Isabel Mebarak Ripoll is a Colombian s...,4690310,True
4,Don Omar,"Reggaeton, latin, Hip-Hop, spanish, latino","William Omar Landrón (born February 10, 1978 i...",1700833,True
...,...,...,...,...,...
78,Imagine Dragons,"indie, indie rock, alternative, rock, indie pop",Imagine Dragons is an American pop band formed...,3943396,True
79,Maroon 5,"pop rock, rock, pop, alternative, alternative ...",Maroon 5 is an American pop rock band that ori...,6434168,True
80,Elton John,"pop, classic rock, singer-songwriter, rock, piano",Sir Elton Hercules John (born Reginald Kenneth...,5072416,True
81,Juanes,"latin, pop, spanish, rock, latino",Juan Esteban Aristizábal Vásquez (born August ...,1170421,True
